In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv
/kaggle/input/m5-point-forecast-valweights/weights_df
/kaggle/input/m5-point-forecast-valweights/__results__.html
/kaggle/input/m5-point-forecast-valweights/custom.css
/kaggle/input/m5-point-forecast-valweights/__notebook__.ipynb
/kaggle/input/m5-point-forecast-valweights/__output__.json
/kaggle/input/m5-point-forecast-createinput-1400/__results__.html
/kaggle/input/m5-point-forecast-createinput-1400/cat_feats.pkl
/kaggle/input/m5-point-forecast-createinput-1400/X_train_df
/kaggle/input/m5-point-forecast-createinput-1400/cat_feats_objs.pkl
/kaggle/input/m5-point-forecast-createinput-1400/y_train.npy
/kaggle/input/m5-point-forecast-createinput-1400/custom.css
/kaggle/input/m5-point-forecast-create

In [2]:
import pickle
from collections import defaultdict
import gc, sys

In [3]:
class Category:
    def __init__(self, unique_items, item_to_id={}):
        self.items = unique_items
        self.item_to_id = item_to_id
        for _id, item_name in enumerate(self.items):
            self.item_to_id[item_name] = _id
        self.type = np.int16
        if len(self.items) > 30000:
            self.type = np.int32
    
    def encode_series(self, items):
        return np.array([self.item_to_id[_item] for _item in items]).astype(self.type)

In [4]:
with open('../input/m5-point-forecast-createinput-1400/X_train_df', 'rb') as f:
    X_train_df = pickle.load(f)
y_train = np.load('../input/m5-point-forecast-createinput-1400/y_train.npy')

In [5]:
with open('../input/m5-point-forecast-valweights/weights_df', 'rb') as f:
    val_weights = pickle.load(f)

In [6]:
val_weights

,item_id,store_id,day_of_first_sale,weight,K,all_zeros,sales_mean,sales_std
0,HOBBIES_1_001,CA_1,902,0.000060,0.855926,0,0.592483,0.849799
1,HOBBIES_1_002,CA_1,144,0.000002,1.278055,0,0.278123,0.586817
2,HOBBIES_1_003,CA_1,1106,0.000013,1.128216,0,0.353160,0.691916
3,HOBBIES_1_004,CA_1,37,0.000063,0.372943,0,1.751599,1.994428
4,HOBBIES_1_005,CA_1,113,0.000029,0.611990,0,1.022778,1.300441
...,...,...,...,...,...,...,...,...
30485,FOODS_3_823,WI_3,3,0.000005,0.863262,0,0.534031,1.177291
30486,FOODS_3_824,WI_3,6,0.000006,0.981145,0,0.374934,0.813791
30487,FOODS_3_825,WI_3,2,0.000027,0.659478,0,0.892726,1.380701
30488,FOODS_3_826,WI_3,940,0.000009,0.677787,0,0.720452,1.249856


In [7]:
X_train_df['y'] = y_train

In [8]:
train_df = X_train_df

In [9]:
train_df = train_df.loc[train_df.day_id >= 1400].reset_index(drop=True)

In [10]:
del X_train_df, y_train
gc.collect()

20

In [11]:
train_df.shape

(15671860, 44)

In [12]:
with open('../input/m5-point-forecast-createinput-1400/cat_feats.pkl', 'rb') as f:
    cat_feats = pickle.load(f)
with open('../input/m5-point-forecast-createinput-1400/cat_feats_name2idx.pkl', 'rb') as f:
    cat_feats_name2idx = pickle.load(f)
with open('../input/m5-point-forecast-createinput-1400/cat_feats_objs.pkl', 'rb') as f:
    cat_feats_objs = pickle.load(f)

In [13]:
col_names = train_df.columns

In [14]:
col_names

Index(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'day_id',
       'week_id', 'year', 'month', 'day_of_month', 'day_of_week', 'snap',
       'sell_price', 'Chanukah End_lag', 'SuperBowl_lag', 'StPatricksDay_lag',
       'LaborDay_lag', 'Pesach End_lag', 'PresidentsDay_lag', 'NewYear_lag',
       'ColumbusDay_lag', 'Ramadan starts_lag', 'Purim End_lag',
       'LentWeek2_lag', 'Thanksgiving_lag', 'IndependenceDay_lag',
       'Christmas_lag', 'NBAFinalsStart_lag', 'Father's day_lag',
       'LentStart_lag', 'EidAlAdha_lag', 'VeteransDay_lag',
       'MartinLutherKingDay_lag', 'Cinco De Mayo_lag', 'Halloween_lag',
       'Easter_lag', 'NBAFinalsEnd_lag', 'OrthodoxChristmas_lag',
       'ValentinesDay_lag', 'MemorialDay_lag', 'Eid al-Fitr_lag',
       'OrthodoxEaster_lag', 'Mother's day_lag', 'y'],
      dtype='object')

In [15]:
(val_weights.weight == 0).sum()

1587

In [16]:
# create historical features

In [17]:
sales_train_val = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')

In [18]:
last_day = 1941

In [19]:
col_names_days = []
for i in range(1,last_day+1):
    col_names_days.append('d_'+str(i))

In [20]:
# 28 day lag
_28dl_dict = {}
for i in range(len(sales_train_val)):
    _row = sales_train_val.iloc[i]
    unit_sales = _row.iloc[6:].values
    _lag = np.zeros(last_day) - 1
    _lag[-len(unit_sales):] = unit_sales
    _28dl_dict[(_row.item_id, _row.store_id)] = _lag.astype(np.float32)
    

In [21]:
_ser = _28dl_dict[('HOBBIES_1_046', 'CA_1')]; _ser

array([-1., -1., -1., ...,  1.,  1.,  1.], dtype=float32)

In [22]:
# 28_day_lag, 7_day_mean, 7_day_std, 28_day_mean, 28_day_std

_28dl_7dm_dict = {}
_28dl_7ds_dict = {}
_28dl_14dm_dict = {}
_28dl_14ds_dict = {}
_28dl_21dm_dict = {}
_28dl_21ds_dict = {}
_28dl_28dm_dict = {}
_28dl_28ds_dict = {}
_count=0
for itst, ser in _28dl_dict.items():
    if _count%1000 == 0:
        print(_count)
    _count+=1
    _28dl_7dm_dict[itst] = pd.Series(ser).rolling(7).mean().values.astype(np.float32)
    _28dl_7ds_dict[itst] = pd.Series(ser).rolling(7).std().values.astype(np.float32)
    _28dl_14dm_dict[itst] = pd.Series(ser).rolling(14).mean().values.astype(np.float32)
    _28dl_14ds_dict[itst] = pd.Series(ser).rolling(14).std().values.astype(np.float32)
    _28dl_21dm_dict[itst] = pd.Series(ser).rolling(21).mean().values.astype(np.float32)
    _28dl_21ds_dict[itst] = pd.Series(ser).rolling(21).std().values.astype(np.float32)
    _28dl_28dm_dict[itst] = pd.Series(ser).rolling(28).mean().values.astype(np.float32)
    _28dl_28ds_dict[itst] = pd.Series(ser).rolling(28).std().values.astype(np.float32)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000


In [23]:
del sales_train_val
gc.collect()

20

In [24]:
def show_progress(progress_str, n_cols=80):
    sys.stdout.write(progress_str + (" " * max(0, n_cols - len(progress_str))) + "\r")
    sys.stdout.flush()

In [25]:
hist_feats = ['_28dl', '_28dl_7dm', '_28dl_7ds', '_28dl_14dm', '_28dl_14ds', '_28dl_21dm', '_28dl_21ds', '_28dl_28dm', '_28dl_28ds']

In [26]:
def item_store_ordering(df_y):
    df_y_gpby_is = df_y.groupby(by=['item_id', 'store_id'])
    #result_df = pd.DataFrame(columns=col_names)
    result_dict = defaultdict(list)
    stats_dict = defaultdict(list)
    for i in range(len(val_weights)):
        if (i%1000) == 0:
            show_progress(f"Processing item_store: {i+1}/{len(val_weights)}")
        _row = val_weights.iloc[i]
        if _row.weight == 0:
            continue
        _item = cat_feats_objs[cat_feats_name2idx['item_id']].item_to_id[_row.item_id]
        _store = cat_feats_objs[cat_feats_name2idx['store_id']].item_to_id[_row.store_id]
        _df = df_y_gpby_is.get_group((_item, _store))
        result_dict['_28dl'].append(_28dl_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_7dm'].append(_28dl_7dm_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_7ds'].append(_28dl_7ds_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_14dm'].append(_28dl_14dm_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_14ds'].append(_28dl_14ds_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_21dm'].append(_28dl_21dm_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_21ds'].append(_28dl_21ds_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_28dm'].append(_28dl_28dm_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        result_dict['_28dl_28ds'].append(_28dl_28ds_dict[(_row.item_id,_row.store_id)][_df.day_id.values-1])
        #_df = _df.loc[(_df.day_id >= _row.day_of_first_sale).values]
        for col in col_names:
            if col == 'y':
                y = _df[col].values
                result_dict[col].append(y)
                # Update stats dict
                #stats_dict['item'].append(_item)
                #stats_dict['store'].append(_store)
                #stats_dict['y_norm_mean'].append(y_norm.mean())
                #stats_dict['y_norm_std'].append(y_norm.std())
                #stats_dict['y_norm_min'].append(y_norm.min())
                #stats_dict['y_norm_max'].append(y_norm.max())
            else:
                result_dict[col].append(_df[col].values)
    for col in col_names:
        result_dict[col] = np.concatenate(result_dict[col])
    for col in hist_feats:
        result_dict[col] = np.concatenate(result_dict[col])
    return pd.DataFrame(result_dict)[list(col_names)+hist_feats]

In [27]:
ordered_train_df = item_store_ordering(train_df)
del train_df
gc.collect()

0

In [28]:
ordered_train_df

,item_id,dept_id,cat_id,store_id,state_id,day_id,week_id,year,month,day_of_month,...,y,_28dl,_28dl_7dm,_28dl_7ds,_28dl_14dm,_28dl_14ds,_28dl_21dm,_28dl_21ds,_28dl_28dm,_28dl_28ds
0,1437,3,1,0,0,1400,11443,2014,11,28,...,0.0,0.0,0.571429,0.534522,0.428571,0.513553,0.380952,0.497613,0.535714,0.744468
1,1437,3,1,0,0,1401,11444,2014,11,29,...,0.0,1.0,0.571429,0.534522,0.500000,0.518875,0.380952,0.497613,0.500000,0.693889
2,1437,3,1,0,0,1402,11444,2014,11,30,...,0.0,1.0,0.571429,0.534522,0.571429,0.513553,0.428571,0.507093,0.535714,0.692935
3,1437,3,1,0,0,1403,11444,2014,12,1,...,0.0,1.0,0.714286,0.487950,0.571429,0.513553,0.476190,0.511766,0.535714,0.692935
4,1437,3,1,0,0,1404,11444,2014,12,2,...,3.0,0.0,0.571429,0.534522,0.571429,0.513553,0.428571,0.507093,0.428571,0.503953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14856137,1436,2,0,9,2,1909,11612,2016,4,20,...,0.0,5.0,2.142857,1.951800,2.142857,1.610406,1.809524,1.661898,1.785714,1.728993
14856138,1436,2,0,9,2,1910,11612,2016,4,21,...,0.0,3.0,2.142857,1.951800,2.357143,1.499084,1.761905,1.609496,1.892857,1.707050
14856139,1436,2,0,9,2,1911,11612,2016,4,22,...,0.0,2.0,2.428571,1.718249,2.357143,1.499084,1.809524,1.600595,1.928571,1.698116
14856140,1436,2,0,9,2,1912,11613,2016,4,23,...,0.0,0.0,2.285714,1.889822,2.214286,1.625687,1.666667,1.622755,1.785714,1.685607


In [29]:
with open('../input/m5-point-forecast-createinput-1400/X_val_df', 'rb') as f:
    X_val_df = pickle.load(f)
y_val = np.load('../input/m5-point-forecast-createinput-1400/y_val.npy')

In [30]:
X_val_df['y'] = y_val

In [31]:
val_df = X_val_df

In [32]:
ordered_val_df = item_store_ordering(val_df)
del val_df, X_val_df, y_val
gc.collect()

0

In [33]:
ordered_train_df.to_pickle('ordered_train_df.pkl')

In [34]:
ordered_val_df.to_pickle('ordered_val_df.pkl')

In [35]:
val_weights.to_pickle('val_weights.pkl')

In [36]:
! ls -alrh

total 1.9G
-rw-r--r-- 1 root root 1.7M Jun 29 18:27 val_weights.pkl
-rw-r--r-- 1 root root  96M Jun 29 18:27 ordered_val_df.pkl
-rw-r--r-- 1 root root 1.8G Jun 29 18:27 ordered_train_df.pkl
---------- 1 root root  14K Jun 29 18:21 __notebook__.ipynb
drwxr-xr-x 6 root root 4.0K Jun 29 18:21 ..
drwxr-xr-x 2 root root 4.0K Jun 29 18:27 .


In [37]:
with open('_28dl_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_dict, f)

In [38]:
with open('_28dl_7dm_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_7dm_dict, f)

In [39]:
with open('_28dl_7ds_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_7ds_dict, f)

In [40]:
with open('_28dl_14dm_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_14dm_dict, f)

In [41]:
with open('_28dl_14ds_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_14ds_dict, f)

In [42]:
with open('_28dl_21dm_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_7dm_dict, f)

In [43]:
with open('_28dl_21ds_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_7ds_dict, f)

In [44]:
with open('_28dl_28dm_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_28dm_dict, f)

In [45]:
with open('_28dl_28ds_dict.pkl', 'wb') as f:
    pickle.dump(_28dl_28ds_dict, f)

In [46]:
! ls -alrh 

total 3.9G
-rw-r--r-- 1 root root 1.7M Jun 29 18:27 val_weights.pkl
-rw-r--r-- 1 root root  96M Jun 29 18:27 ordered_val_df.pkl
-rw-r--r-- 1 root root 1.8G Jun 29 18:27 ordered_train_df.pkl
---------- 1 root root  14K Jun 29 18:21 __notebook__.ipynb
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_7ds_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_7dm_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_28ds_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_28dm_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_21ds_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_21dm_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_14ds_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_14dm_dict.pkl
drwxr-xr-x 6 root root 4.0K Jun 29 18:21 ..
drwxr-xr-x 2 root root 4.0K Jun 29 18:27 .


In [47]:
with open('hist_feats.pkl', 'wb') as f:
    pickle.dump(hist_feats, f)

In [48]:
! ls -alrh 

total 3.9G
-rw-r--r-- 1 root root 1.7M Jun 29 18:27 val_weights.pkl
-rw-r--r-- 1 root root  96M Jun 29 18:27 ordered_val_df.pkl
-rw-r--r-- 1 root root 1.8G Jun 29 18:27 ordered_train_df.pkl
-rw-r--r-- 1 root root  154 Jun 29 18:27 hist_feats.pkl
---------- 1 root root  14K Jun 29 18:21 __notebook__.ipynb
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_7ds_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_7dm_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_28ds_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_28dm_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_21ds_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_21dm_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_14ds_dict.pkl
-rw-r--r-- 1 root root 229M Jun 29 18:27 _28dl_14dm_dict.pkl
drwxr-xr-x 6 root root 4.0K Jun 29 18:21 ..
drwxr-xr-x 2 root root 4.0K Jun 29 18:27 .
